In [1]:
# # Requerimientos de librerías:
# !pip install spacy
# !python -m spacy download es_core_news_md

# !pip install transformers
# !pip install transformers scipy ftfy accelerate

# !git clone https://github.com/JorgeSauri/COMSINT.git 

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
import numpy as np
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
from recomendaciones_comsint import Recomendador

c:\Users\jsaur\anaconda3\envs\ENV1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Con tamaño embeddings 256
- CNN de max 512 kernels (v4)

In [4]:
version = 4

recomendador256 = Recomendador(fuente='recetario_mexicano_small.csv',
                            nutricion = 'nutricion.csv',
                            canasta='canasta_basica.csv')

recomendador256.CargarModelo(emb_size=256, version=version)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
No se encontró el modelo Modelos/Modelo_Nut_FV_DistilBERT_04_EMBED-256_CNN.h5
Puedes crear uno nuevo con el método EntrenarModelo()



In [5]:
recomendador256.generar_dataset_entrenamiento_nut(df_nutricionales='nutricion.csv', 
                                                  min_ingredientes=5, max_ingredientes=11,
                                                  min_unidades=1, max_unidades=11,
                                                  numero_recetas=1)

Generando 1  recetas aleatorias...



100%|██████████| 1/1 [00:00<00:00,  5.60it/s]


array([['4 piezas de cordero  frito  cocinado  cerebro  carnes de variedades y subproductos, 10 piezas de keebler  suave  galleta de pasas de avena  tesoros de panaderos, 9 gramos de ternera  cruda  separable solamente  hueso  frente  vástago  australiano, 10 cucharadas de ternera  frita  cocinada  hígado  carnes de variedades y subproductos, 3 gramos de frijoles  sólidos drenados  paquete regular  enlatado  verde  chasquido, 5 tazas de suero  fluido  dulce',
        '5304.28', '691.19', '130.78', '228.45']], dtype='<U452')

In [6]:
recomendador256.NUM_RECETAS = 10000
recomendador256.EMB_SIZE = 256
ITER = 1
INITIAL_EPOCH = 0
EPOCHS = 100

Histories = []
for iteracion in range(ITER):
  MINU = 1 
  MAXU = 11   
  print('\nITERACIÓN:', iteracion+1)
  print('min unidades:',MINU, ' max unidades:', MAXU)
  print('Entrenando desde epoch', INITIAL_EPOCH)
  print('------------------------------------------\n')
  modelo256, history = recomendador256.EntrenarModelo(df_nutricionales='nutricion.csv',
                                #df_training='recipes_con_info_nutricional.csv',
                                df_test='recetas_test.csv', 
                                df_val='recetas_val.csv',
                                learning_rate=1e-4,
                                version=version, 
                                initial_epoch = INITIAL_EPOCH,                             
                                epochs=INITIAL_EPOCH + EPOCHS, 
                                kernels=128,                                             
                                min_ingredientes=5, max_ingredientes=11,
                                min_unidades=MINU, max_unidades=MAXU,                               
                                save=True, verbose=True)
  INITIAL_EPOCH = history.epoch[-1]

  Histories.append(history)



ITERACIÓN: 1
min unidades: 1  max unidades: 11
Entrenando desde epoch 0
------------------------------------------

Error al cargar archivos NumPy.
datasets/numpy/10000_recetas_random_EMBED-256_DATA_X.npy no existe o está corrupto.
datasets/numpy/10000_recetas_random_EMBED-256_DATA_Y.npy no existe o está corrupto.
Generando 10000  recetas aleatorias...



100%|██████████| 10000/10000 [00:14<00:00, 672.02it/s]


Calculando vector de características de 10000 recetas...


100%|██████████| 10000/10000 [9:00:25<00:00,  3.24s/it] 


Error al cargar archivos NumPy.
datasets/numpy/9_TEST_EMBED-256_DATA_X.npy no existe o está corrupto.
datasets/numpy/9_TEST_EMBED-256_DATA_Y.npy no existe o está corrupto.
Procesando dataset de testing...
Procesando recetas_test.csv


...

Error al cargar archivos NumPy.
datasets/numpy/7_VAL_EMBED-256_DATA_X.npy no existe o está corrupto.
datasets/numpy/7_VAL_EMBED-256_DATA_Y.npy no existe o está corrupto.
Procesando dataset de validación...
Procesando recetas_val.csv


100%|██████████| 7/7 [00:00<00:00, 7013.89it/s]


Calculando vector de características de 7 recetas...


100%|██████████| 7/7 [00:23<00:00,  3.36s/it]


Model: "ModeloCNNNut_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 CapaEntrada (InputLayer)    [(None, 196608)]          0         
                                                                 
 RESHAPING (Reshape)         (None, 256, 768)          0         
                                                                 
 batch_normalization (BatchN  (None, 256, 768)         3072      
 ormalization)                                                   
                                                                 
 CONV_1 (Conv1D)             (None, 252, 512)          1966592   
                                                                 
 POOLING_1 (MaxPooling1D)    (None, 251, 512)          0         
                                                                 
 CONV_2 (Conv1D)             (None, 249, 256)          393472    
                                                  

In [ ]:
for i in range(len(Histories)):
    history = Histories[i]    
    pd.DataFrame(history.history).plot()
    plt.title('ITERACIÓN ' + str(i))
    plt.show()
    print('LOSS:',history.history['loss'][-1], ' -- MAE:', history.history['mae'][-1], 
          ' -- VAL_LOSS:', history.history['val_loss'][-1], ' -- VAL_MAE:', history.history['val_mae'][-1])
    print('----------------------------------------------------------')

: 

In [ ]:
modelo256, history2 = recomendador256.EntrenarModelo(df_nutricionales='nutricion.csv',
                            df_training='recipes_con_info_nutricional.csv',
                            df_test='recetas_test.csv', 
                            df_val='recetas_val.csv',
                            learning_rate=1e-4,
                            version=version, 
                            initial_epoch = INITIAL_EPOCH,                             
                            epochs=INITIAL_EPOCH + EPOCHS, 
                            kernels=128,                                             
                            min_ingredientes=5, max_ingredientes=11,
                            min_unidades=MINU, max_unidades=MAXU,                               
                            save=True, verbose=True)

Histories.append(history)

: 

In [ ]:
for i in range(len(Histories)):
    history = Histories[i]    
    pd.DataFrame(history.history).plot()
    plt.title('ITERACIÓN ' + str(i))
    plt.show()
    print('LOSS:',history.history['loss'][-1], ' -- MAE:', history.history['mae'][-1], 
          ' -- VAL_LOSS:', history.history['val_loss'][-1], ' -- VAL_MAE:', history.history['val_mae'][-1])
    print('----------------------------------------------------------')

: 